In [ ]:
#Loads all packages at beginning
import os
import shutil

import numpy as np
import pandas as pd
import pickle as pkl

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

import sys
import os
import getpass

import matplotlib.pyplot as plt
import networkx as nx
from sklearn.covariance import GraphicalLassoCV

import time

# Setup

In [ ]:
username = getpass.getuser()
if username == 'mickey':
    data_directory = '/Volumes/Samsung_T5/aibs_data/example_ecephys_project_cache/'
    save_directory = '/Volumes/Samsung_T5/aibs_data/savedir/'
elif username == 'rahul':
    data_directory = 'D:\\OneDrive - UW\\research\\data@aibs' # must be updated to a valid directory in your filesystem
    save_directory = 'D:\\OneDrive - UW\\research\\data@aibs\\savedir'
elif username == 'Ryan':
    data_directory = 'C:\\Users\\Ryan\\Documents\\Grad School\\Research\\Eli\\Group\\Hack\\Allen_Data' # must be updated to a valid directory in your filesystem
    save_directory = 'C:\\Users\\Ryan\\Documents\\Grad School\\Research\\Eli\\Group\\Hack\\Save_Data'
    
    
manifest_path = os.path.join(data_directory, "manifest.json")

In [ ]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [ ]:
# sessions[sessions.session_type == 'functional_connectivity'].index

In [ ]:
# Load Sessions and corresponding data
sessions = cache.get_session_table()
probes = cache.get_probes()
channels = cache.get_channels()
units = cache.get_units()


#print('Total number of sessions: ' + str(len(sessions)))
#sessions.head()

In [ ]:
# sessions.ecephys_structure_acronyms.values[44]
# sessions[sessions.session_type == 'functional_connectivity'].index.size

## Function Definitions

In [ ]:
class A_Config:
    def __init__(self, data_dir, save_dir, sess_id, stimulus):
        
        self.data_dir = data_dir 
        self.save_dir = save_dir
        self.sess_id = sess_id
        self.stimulus = stimulus

        self.name = os.path.join(self.save_dir,"ID{}_{}".format(self.sess_id, self.stimulus))

In [ ]:
def save_Ps(cache,sess_id,stim_label_list,bin_size):
    data = cache.get_session_data(sess_id)
    #t_bin = {}
    #P = {}
    for i, stim_label in enumerate(stim_label_list):
        allencon = A_Config(data_directory,save_directory,sess_id,stim_label)
        print(i)
        # get start, end time points
        stim_data_table = data.get_stimulus_table()[data.get_stimulus_table().stimulus_name==stim_label]
        start_time = np.array(stim_data_table.start_time)
        stop_time = np.array(stim_data_table.stop_time)
        trial_time = np.mean(stop_time - start_time)
        bin_edges = np.arange(trial_time + bin_size, step = bin_size )
        t_bin = bin_edges
        print(i)
        stim_presentation_ids = data.stimulus_presentations.loc[
            (data.stimulus_presentations['stimulus_name'] == stim_label)
            ].index.values
        print(i)
        unit_ids = data.units.index.values
        print(i)
        P = data.presentationwise_spike_counts(bin_edges,stim_presentation_ids,unit_ids)/bin_size
        pkl.dump(P,open(allencon.name+'_bin_{}_P.p'.format(bin_size),'wb'))
        pkl.dump(t_bin,open(allencon.name+'_bin_{}_t.p'.format(bin_size),'wb'))

In [ ]:
def plot_matrix(mat,motif,id):
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    fig,ax = plt.subplots(1,1,figsize=(10,10))
    norm = mpl.colors.Normalize(vmin=0.,vmax=np.max(mat))
    img=ax.matshow(mat,cmap='RdBu_r',norm=norm)
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    cbar_ax.tick_params(labelsize=20)
    fig.colorbar(img,cax=cbar_ax)
    ax.set_ylabel("Neuron index",fontsize=20)
    ax.set_xlabel("Neuron index",fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=20)
    fig.savefig(id+'_'+str(motif)+'.pdf',format='pdf',dpi=600)
    fig.savefig(id+'_'+str(motif)+'.png',format='png',dpi=600)

In [ ]:
def smooth_spikes( P, kernel_width , bin_size ):
    # Rahul
    """
    P: #trials x #units x #time-points sized ndarray
    kernel_param: width of kernel (in samplingrate*1000 units)

    """

    import time
    import scipy.stats, scipy.signal
    import scipy as sp

    start = time.time() # tic Measures function running speed. Works only on Macs

    units = P.shape[0]
    trials = P.shape[1]
    bins = P.shape[2]

    # Create fake data for testing purposes
    #    array = np.random.randint(0,2,(trials,bins))
    #    sigma = 45

    #Define kernel
    kernel_param = kernel_width/bin_size
    sigma = kernel_param/6.
    edges = np.arange(-3*sigma,3*sigma+1,1)
    kernel = sp.stats.norm.pdf(edges,0, sigma) #Use a gaussian function
    kernel = kernel/sum(kernel)
    
    #Compute Spike Density Function for all trials
    #X = np.zeros((trials,units,bins))
    X = sp.signal.convolve(P, kernel[None,:, None], mode='same',method='direct')

    print('Run time for SDF function was ' + str(round(time.time()-start)) + 'seconds') # toc
    return X

## Run Calculations (Run for each stimulus/session)

In [ ]:
sess_id=791319847
stim_label_list = ['natural_scenes','drifting_gratings_contrast','drifting_gratings'] #All stimuli used
stim_label_list_idx = 2
stim_label = stim_label_list[stim_label_list_idx]

allencon = A_Config(data_directory,save_directory,sess_id,stim_label)
print(stim_label)
print(allencon.stimulus)

In [ ]:
#Bin & Kernel Parameters
bin_size=0.01
kernel_width=0.5

#kernel_param=kernel_width/bin_size

In [ ]:
save_Ps(cache, sess_id, stim_label_list = [stim_label], bin_size=bin_size)

In [ ]:
X={}
for i, stim_label_ in enumerate([stim_label]):
    print(i)
    allencon = A_Config(data_directory,save_directory,sess_id,stim_label_)
    P = pkl.load(open(allencon.name+'_bin_{}_P.p'.format(bin_size),'rb'))
    X[stim_label] = smooth_spikes(P.values,kernel_width,bin_size)

In [ ]:
X[stim_label][0,:,:].shape

In [ ]:
f = plt.figure()
# for p_idx, k in enumerate([0,9,19,29]):
#     ax = f.add_subplot(2,2,p_idx+1)
#     plt.plot(X[stim_label][0,:,k])
#     plt.xlabel('samples')
#     plt.ylabel('firing rate')
    
plt.imshow(X[stim_label][0,:,:])

In [ ]:
a = np.zeros(n_trial)
for k in range(n_trial):
    a[k] = np.mean(np.std(X[stim_label][k,:,:],axis=0)==0)

np.mean(a)

In [ ]:
n_trial = X[stim_label].shape[0]
n_time = X[stim_label].shape[1]
n_unit = X[stim_label].shape[2]
dither_noise = 0.1
adj_mats = []
unit_list = []
for k in range(1):
    print(k)
    t_ = time.time()
    units_to_use_idx_ = np.std(X[stim_label][k,:,:],axis=0)!=0
    model = GraphicalLassoCV()
    model.fit(X[stim_label][k,:,units_to_use_idx_])
    cov_ = model.covariance_
    prec_ = model.precision_
    adj_mats.append(prec_)
    unit_list.append(units_to_use_idx_)
    print('{}s'.format(time.time()-t_))

In [ ]:

plot_matrix(100*np.abs(prec_),'{}_{}_trial{}'.format(sess_id,stim_label,0),'ggm')

In [ ]:
X[stim_label][k,:,units_to_use_idx_].shape

# Additional Calculations/Methods

In [ ]:
X={}

In [ ]:
sum(P[stim_label].values)

In [ ]:
X[stim_label] = smooth_spikes(P[stim_label].values,kernel_param)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(X[stim_label][0,0,:])
plt.plot(X[stim_label][0,1,:])
plt.plot(X[stim_label][0,2,:])
plt.plot(X[stim_label][0,3,:])
plt.plot(X[stim_label][0,4,:])

In [ ]:
def load_sess_trials(cache, sess_id, stim_label_list, bin_size, kernel_width):
    # load session data
    data = cache.get_session_data(sess_id)
    t_bin = {}
    P = {}
    X = {}
    for i, stim_label in enumerate(stim_label_list):
        print(i)
        # get start, end time points
        stim_data_table = data.get_stimulus_table()[data.get_stimulus_table().stimulus_name==stim_label]
        start_time = np.array(stim_data_table.start_time)
        stop_time = np.array(stim_data_table.stop_time)
        trial_time = np.mean(stop_time - start_time)
        bin_edges = np.arange(trial_time + bin_size, step = bin_size )
        t_bin[stim_label] = bin_edges
        print(i)
        stim_presentation_ids = data.stimulus_presentations.loc[
            (data.stimulus_presentations['stimulus_name'] == stim_label)
            ].index.values
        print(i)
        unit_ids = data.units.index.values
        print(i)
        P[stim_label] = data.presentationwise_spike_counts(bin_edges,stim_presentation_ids,unit_ids)/bin_size
        print(i)
        # get windowed spike times
        # S[i] = window_spk_times(data,start_time,stop_time)
        # get binned spike rates
        # t_bin[i], P[i] = bin_spikes(S[i],bin_size)
        # kernel smoothed estimate of binned spike rates
        X[stim_label] = smooth_spikes(P[stim_label].values,kernel_width)
        
    return (t_bin, P, X)

In [ ]:
import numpy as np
import pandas as pd

data = cache.get_session_data(sess_id)
t_bin = {}
#P = {}
for i, stim_label in enumerate(stim_label_list):
    allencon = A_Config(data_directory,save_directory,sess_id,stim_label)
    print(i)
    # get start, end time points
    stim_data_table = data.get_stimulus_table()[data.get_stimulus_table().stimulus_name==stim_label]
    start_time = np.array(stim_data_table.start_time)
    stop_time = np.array(stim_data_table.stop_time)
    trial_time = np.mean(stop_time - start_time)
    bin_edges = np.arange(trial_time + bin_size, step = bin_size )
    t_bin[stim_label] = bin_edges
    print(i)
    stim_presentation_ids = data.stimulus_presentations.loc[
        (data.stimulus_presentations['stimulus_name'] == stim_label)
        ].index.values
    print(i)
    unit_ids = data.units.index.values
    print(i)
    P = data.presentationwise_spike_counts(bin_edges,stim_presentation_ids,unit_ids)/bin_size
    pkl.dump(P,open(allencon.name+'_bin_{}_P.p'.format(bin_size),'wb'))
    print(i)
    # get windowed spike times
    # S[i] = window_spk_times(data,start_time,stop_time)
    # get binned spike rates
    # t_bin[i], P[i] = bin_spikes(S[i],bin_size)
    # kernel smoothed estimate of binned spike rates


In [ ]:
P=pkl.load(open(allencon.name+'_bin_{}_P.p'.format(bin_size),'rb'))
X = smooth_spikes(P.values,kernel_width=0.1, bin_size= 0.01)

In [ ]:
X[0,:,:]

In [ ]:
t_bin,P,X = load_sess_trials(cache, 794812542, stim_label_list = ['drifting_gratings_contrast'], bin_size=0.01, kernel_param=0.1)

In [ ]:
dir()

In [ ]:
data_dg=data.get_stimulus_table()[data.get_stimulus_table().stimulus_name=='drifting_gratings_contrast']

In [ ]:
data_dg

In [ ]:
data.spike_times

In [ ]:
data.units.index

In [ ]:
data.spike_times[data.units.index[0]]

In [ ]:
unit_spikes=data.spike_times[data.units.index[0]]

In [ ]:
unit_spikes[np.logical_and(unit_spikes>=data_dg.start_time.values[0],unit_spikes<data_dg.start_time.values[1])]

In [ ]:
def smooth_spikes( P, kernel_width , bin_size ):
    # Rahul
    """
    P: #trials x #units x #time-points sized ndarray
    kernel_param: width of kernel (in samplingrate*1000 units)

    """

    import time
    import scipy.stats, scipy.signal
    import scipy as sp

    start = time.time() # tic Measures function running speed. Works only on Macs

    trials = P.shape[0]
    units = P.shape[1]
    bins = P.shape[2]

    # Create fake data for testing purposes
    #    array = np.random.randint(0,2,(trials,bins))
    #    sigma = 45

    #Define kernel
    kernel_param = kernel_width/bin_size
    sigma = kernel_param/6.
    edges = np.arange(-3*sigma,3*sigma+1,1)
    kernel = sp.stats.norm.pdf(edges,0, sigma) #Use a gaussian function
    kernel = kernel/sum(kernel)
    
    #Compute Spike Density Function for all trials
    #X = np.zeros((trials,units,bins))
    X = sp.signal.convolve(P, kernel[None,None, :], mode='same',method='direct')

    print('Run time for SDF function was ' + str(round(time.time()-start)) + 'seconds') # toc
    return X

In [ ]:
import scipy as sp

In [ ]:
kernel_width = 0.01
sigma = kernel_width/6.
edges = np.arange(-3*sigma,3*sigma+.001,.001)
kernel = sp.stats.norm.pdf(edges,0, sigma) #Use a gaussian function

In [ ]:
kernel_width = .01
bin_size = .001
kernel_param = kernel_width/bin_size
sigma = kernel_param/6.
edges = np.arange(-3*sigma,3*sigma+1,1)
kernel = sp.stats.norm.pdf(edges,0, sigma) #Use a gaussian function
kernel = kernel/sum(kernel)

In [ ]:
plt.plot(edges,kernel/sum(kernel))
print(sum(kernel))

In [ ]:
sum(kernel)

In [ ]:
def est_frate(P_stim, k_p):
    """
    P_stim: #trials x #units x #time-points sized ndarray
    k_p: width of kernel (in samplingrate*1000 units)

    """

    import numpy as np
    import time
    import scipy.stats, scipy.signal
    import scipy as sp

    start = time.time() # tic Measures function running speed. Works only on Macs

    trials = P_stim.shape[0]
    units = P_stim.shape[1]
    bins = P_stim.shape[2]

    # Create fake data for testing purposes
    #    array = np.random.randint(0,2,(trials,bins))
    #    sigma = 45

    #Define kernel
    sigma = k_p/6. 
    edges = np.arange(-3*sigma,3*sigma+.001,.001)
    kernel = sp.stats.norm.pdf(edges,0, sigma) #Use a gaussian function

    #Compute Spike Density Function for all trials
    Sdf = np.zeros((trials,units,bins))
    Sdf = sp.signal.convolve(P_stim, kernel[None,None, :], mode='same',method='direct')

    print('Run time for SDF function was ' + str(round(time.time()-start)) + 'seconds') # toc

    return Sdf

In [ ]:
P_stim = np.random.rand(3,5,100)

In [ ]:
P_stim

In [ ]:
X_stim=est_frate(P_stim,0.01)

In [ ]:
X_stim

In [ ]:
plt.plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(P_stim[0,0,:])


In [ ]:
plt.plot(X_stim[0,0,:])

In [ ]:
plt.plot(P_stim[0,1,:])

In [ ]:
plt.plot(X_stim[0,1,:])

In [ ]:
plt.plot(P_stim[1,1,:])

In [ ]:
plt.plot(X_stim[1,1,:])

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl

def save_Ps(cache,sess_id,stim_label_list,bin_size):
    data = cache.get_session_data(sess_id)
    #t_bin = {}
    #P = {}
    for i, stim_label in enumerate(stim_label_list):
        allencon = A_Config(data_directory,save_directory,sess_id,stim_label)
        print(i)
        # get start, end time points
        stim_data_table = data.get_stimulus_table()[data.get_stimulus_table().stimulus_name==stim_label]
        start_time = np.array(stim_data_table.start_time)
        stop_time = np.array(stim_data_table.stop_time)
        trial_time = np.mean(stop_time - start_time)
        bin_edges = np.arange(trial_time + bin_size, step = bin_size )
        t_bin = bin_edges
        print(i)
        stim_presentation_ids = data.stimulus_presentations.loc[
            (data.stimulus_presentations['stimulus_name'] == stim_label)
            ].index.values
        print(i)
        unit_ids = data.units.index.values
        print(i)
        P = data.presentationwise_spike_counts(bin_edges,stim_presentation_ids,unit_ids)/bin_size
        pkl.dump(P,open(allencon.name+'_bin_{}_P.p'.format(bin_size),'wb'))
        pkl.dump(t_bin,open(allencon.name+'_bin_{}_t.p'.format(bin_size),'wb'))